In [1]:
%autosave 0

from WindPy import *
w.start()
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import backtrader as bt
from backtrader.feeds import GenericCSVData
import datetime
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import akshare as ak
import quantstats as qs
import warnings
import pyfolio as pf
import pdb
import os
from scipy import linalg
from statsmodels.api import tsa
warnings.filterwarnings('ignore')
from functools import reduce
from scipy.signal import savgol_filter

today_time1 = datetime.date.today().strftime("%Y-%m-%d")
today_time2 = datetime.date.today().strftime("%Y%m%d")

print(today_time1)
print(today_time2)

pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_rows', None)
plt.rcParams['font.family']='sans-serif'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.sans-serif'] = ['SimHei']

Autosave disabled
Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=Tr

2023-04-26
20230426


C:\Users\Administrator\Anaconda3\lib\site-packages\pyfolio\pos.py:27: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; mutltipliers will not be applied' +


In [2]:
#导入数据，并分月度、周度、日度数据分别处理
#1、注意有一部分数据是通过两个excel导入的，excel需要更新； 2、注意时间轴是如何移动的，考虑数据可得性，将月度数据滞后20天，周度、日度数据滞后1天； 3、数据进行了标准化

In [3]:
dataImport1=w.edb("M0017126,M0017128,M0017129,M0000545,M0000607,S0073300,S6114593,S0179656,S0179664,S0143412,S0031550,S0000066,M5525763,M5525764,M5525769,M0001383,M0001385,M0009973,M0009974,M0009975,M0009976,M0057874,M0057875,M0009977,M0057876,M0057877,M1010885,M1010906,M0329545,M0048486,M0048490,M1004263,M1004267,M5567876,S0059749", "2007-01-01", today_time1,"")


dates1 = pd.to_datetime(dataImport1.Times)
df1 = pd.DataFrame(dataImport1.Data,index=['PMI','PMI：新订单','PMI：新出口订单','工业增加值：当月同比','出口金额：当月同比'\
                                ,'商品房销售面积：累计同比','销量：汽车：当月同比','铜现货','螺纹现货','热卷现货'\
                                ,'BDI','CCFI','社会融资规模存量：同比','社会融资规模存量：人民币贷款：同比'\
                                ,'社会融资规模存量：企业债券：同比','M1：同比','M2：同比','人民币贷款','短期人民币贷款'\
                                ,'中长期人民币贷款','居民部门贷款','居民部门短期贷款','居民部门长期贷款','企业贷款'\
                                ,'企业短期贷款','企业中期贷款','存单利率AAA1Y','存单利率AA1Y','MLF利率：1年','利率互换：1年'\
                                ,'利率互换：5年','国开债到期收益率：1年','国开债到期收益率：5年','GDP：现价'\
                                ,'中债国债到期收益率：10年'],columns = dates1).T




In [4]:
df1= df1.fillna(method='ffill')
df1['人民币贷款占GDP比']=df1['人民币贷款']/df1['GDP：现价']
df1['短期人民币贷款占GDP比']=df1['短期人民币贷款']/df1['GDP：现价']
df1['中长期人民币贷款占GDP比']=df1['中长期人民币贷款']/df1['GDP：现价']
df1['居民部门贷款占GDP比']=df1['居民部门贷款']/df1['GDP：现价']
df1['居民部门短期贷款占GDP比']=df1['居民部门短期贷款']/df1['GDP：现价']
df1['居民部门长期贷款占GDP比']=df1['居民部门长期贷款']/df1['GDP：现价']
df1['企业贷款占GDP比']=df1['企业贷款']/df1['GDP：现价']
df1['企业短期贷款占GDP比']=df1['企业短期贷款']/df1['GDP：现价']
df1['企业中期贷款占GDP比']=df1['企业中期贷款']/df1['GDP：现价']
df1['存单利率AAA1Y与MLF利率利差']=df1['存单利率AAA1Y']-df1['MLF利率：1年']
df1['利率互换FR007与国开到期收益率价差1Y']=df1['利率互换：1年']-df1['国开债到期收益率：1年']
df1['利率互换FR007与国开到期收益率价差5Y']=df1['利率互换：5年']-df1['国开债到期收益率：5年']


df2=df1[['PMI','PMI：新订单','PMI：新出口订单','工业增加值：当月同比','出口金额：当月同比','商品房销售面积：累计同比'\
         ,'销量：汽车：当月同比','社会融资规模存量：同比','社会融资规模存量：人民币贷款：同比','社会融资规模存量：企业债券：同比'\
         ,'M1：同比','M2：同比','人民币贷款占GDP比','短期人民币贷款占GDP比','中长期人民币贷款占GDP比','居民部门贷款占GDP比'\
         ,'居民部门短期贷款占GDP比','居民部门长期贷款占GDP比','企业贷款占GDP比','企业短期贷款占GDP比','企业中期贷款占GDP比'\
         ,'利率互换FR007与国开到期收益率价差1Y','利率互换FR007与国开到期收益率价差5Y']]
df2.index = df2.index + pd.DateOffset(days=20)
#df2.index = df2.index + pd.DateOffset(months=1)
df2 = df2.resample('1W').mean()
print(df2)

df3=df1[['铜现货','螺纹现货']]
df3.index = df3.index + pd.DateOffset(days=1)
df3= df3.fillna(method='ffill')
df3 = df3.resample('1W').mean()

df4=df1.drop(columns=['PMI','PMI：新订单','PMI：新出口订单','工业增加值：当月同比','出口金额：当月同比','商品房销售面积：累计同比'\
                      ,'销量：汽车：当月同比','社会融资规模存量：同比','社会融资规模存量：人民币贷款：同比'\
                      ,'社会融资规模存量：企业债券：同比','M1：同比','M2：同比','人民币贷款占GDP比','短期人民币贷款占GDP比'\
                      ,'中长期人民币贷款占GDP比','居民部门贷款占GDP比','居民部门短期贷款占GDP比','居民部门长期贷款占GDP比'\
                      ,'企业贷款占GDP比','企业短期贷款占GDP比','企业中期贷款占GDP比','利率互换FR007与国开到期收益率价差1Y'\
                      ,'利率互换FR007与国开到期收益率价差5Y','铜现货','螺纹现货'])
df4 = df4.resample('1W').mean()


data_all1=pd.concat([df2,df3,df4],axis=1,join='outer')
data_all1['中债国债到期收益率:10年（T+1）']=data_all1['中债国债到期收益率：10年'].shift(-1)
#df3= df2.fillna(method='bfill')
print(data_all1)
data_all1.to_excel('./data_all1.xlsx')


                  PMI    PMI：新订单  PMI：新出口订单  工业增加值：当月同比   出口金额：当月同比  \
2007-01-21        NaN        NaN        NaN         NaN         NaN   
2007-01-28        NaN        NaN        NaN         NaN         NaN   
2007-02-04        NaN        NaN        NaN         NaN         NaN   
2007-02-11        NaN        NaN        NaN         NaN         NaN   
2007-02-18        NaN        NaN        NaN         NaN         NaN   
2007-02-25  55.100000  59.600000  55.200000   24.710000   33.240000   
2007-03-04  55.100000  59.600000  55.200000   24.710000   33.240000   
2007-03-11  55.100000  59.600000  55.200000   24.710000   33.240000   
2007-03-18  55.100000  59.600000  55.200000   24.710000   33.240000   
2007-03-25  53.500000  57.600000  53.440000   15.022000   48.104000   
2007-04-01  53.100000  57.100000  53.000000   12.600000   51.820000   
2007-04-08  53.100000  57.100000  53.000000   12.600000   51.820000   
2007-04-15  53.100000  57.100000  53.000000   12.600000   51.820000   
2007-0

In [21]:
#月度
dataImport2=w.edb("S0073285,S0028207,M0000559,M0000557,M0000548,M0068071,M0017136,M0017127,M0017129,M0017134,M5541327,S0028199,M5207849,S5065106,M0001699,M5525763,M0001385", "2007-01-01", today_time1,"")
dates2 = pd.to_datetime(dataImport2.Times)
df5 = pd.DataFrame(dataImport2.Data,index=['本年土地成交价款:累计同比','产量:家用电冰箱:当月同比','工业企业:亏损企业亏损额:累计同比'\
                                ,'工业企业:利润总额:累计同比','工业增加值:国有及国有控股企业:当月同比','工业增加值:汽车制造业:当月同比'\
                                ,'PMI:从业人员','PMI:生产','PMI:新出口订单','PMI:主要原材料购进价格'\
                                ,'社会融资规模:企业债券净融资:当月值:初值','产量:彩电:当月同比','银行结售汇差额:当月值'\
                                ,'平均批发价:猪肉','货币当局:政府存款','社会融资规模存量:同比','M2:同比'],columns = dates2).T



#rawdata1 = pd.read_excel('./数据底稿重建.xlsx',sheet_name='Sheet3',header=0,encoding="gbk")
rawdata1 = pd.read_excel('./数据底稿重建.xlsx',sheet_name='Sheet3',header=0,engine='openpyxl')
df8=rawdata1.set_index('时间')

rawdata = pd.read_excel('./量化指标补充.xlsx',sheet_name='地方债净供给',header=0,engine='openpyxl')
df7=rawdata.set_index('时间')

#日度
dataImport3=w.edb("M0017141,M0017140,S0105897,M0220163,M0041653,M1005158,M0329545,M1004271,M0041371,M0330244,M0041739", "2007-01-01", today_time1,"")
dates3 = pd.to_datetime(dataImport3.Times)
df6 = pd.DataFrame(dataImport3.Data,index=['SHIBOR:1个月','SHIBOR:2周','南华工业品指数','存款类机构质押式回购加权利率:7天'\
                                           ,'银行间质押式回购加权利率:7天','同业存单到期收益率:1年','MLF:1年','中债国开债到期收益率:10年'\
                                           ,'逆回购利率:7天','成交量:R001','成交量:银行间质押式回购'],columns = dates3).T
#print(df1)
#df6.index = df6.index + pd.DateOffset(days=1)
df5 = df5.dropna()
df7 = df7.dropna()
df8 = df8.dropna()
df6 = df6.dropna()

data_all2=pd.concat([df5,df7,df8],axis=1,join='outer')
data_all2.index = data_all2.index + pd.DateOffset(days=20)
#data_all2.index = data_all2.index + pd.DateOffset(months=1)
#df3= df2.fillna(method='ffill')

data_all3=pd.concat([data_all2,df6],axis=1,join='outer')
data_all3 = data_all3.resample('1W').mean()


data_all3 = data_all3.fillna(method='ffill')

data_all3['社融-M2']=data_all3['社会融资规模存量:同比']-data_all3['M2:同比']
data_all3['DR007-政策利率']=data_all3['存款类机构质押式回购加权利率:7天']-data_all3['逆回购利率:7天']
data_all3['同业存单利率-政策利率']=data_all3['同业存单到期收益率:1年']-data_all3['MLF:1年']
data_all3['隐含税率']=1-data_all1['中债国债到期收益率：10年']/data_all3['中债国开债到期收益率:10年']
data_all3['隔夜成交占比']=data_all3['成交量:R001']/data_all3['成交量:银行间质押式回购']
data_all4=pd.concat([data_all1,data_all3],axis=1,join='outer')

data_all4.to_excel('data_all4.xlsx')

In [22]:
#进行滤波处理
data_all4_smoothed1 = pd.DataFrame()
for col in data_all4.columns:
    # 将指定列中的空值删除，然后再进行平滑处理
    col_data = data_all4[col].dropna(axis=0)
    smoothed_data = savgol_filter(col_data, window_length=11, polyorder=3)
    # 将平滑后的数据存储到DataFrame对象中
    data_all4_smoothed1[col] = pd.Series(smoothed_data, index=col_data.index)
data_all4_smoothed1

,PMI,PMI：新订单,PMI：新出口订单,工业增加值：当月同比,出口金额：当月同比,商品房销售面积：累计同比,销量：汽车：当月同比,社会融资规模存量：同比,社会融资规模存量：人民币贷款：同比,社会融资规模存量：企业债券：同比,...,MLF:1年,中债国开债到期收益率:10年,逆回购利率:7天,成交量:R001,成交量:银行间质押式回购,社融-M2,DR007-政策利率,同业存单利率-政策利率,隐含税率,隔夜成交占比
2007-02-25,55.281219,59.825974,55.456484,25.662609,31.371203,NaN,36.784332,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-04,55.166533,59.682617,55.330330,24.968190,32.436629,NaN,35.638625,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-11,54.826873,59.259041,54.852807,23.174583,35.928937,NaN,32.801862,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-18,54.368665,58.687429,54.229297,20.702624,40.573086,NaN,28.864270,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-03-25,53.898335,58.099967,53.665181,17.973144,45.094037,0.077423,24.416076,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04-01,53.522311,57.628838,53.365841,15.406978,48.216751,0.007293,20.047508,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04-08,53.569264,57.681152,54.080693,14.012226,45.630247,1.171129,16.964605,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04-15,53.838328,57.994605,55.314326,13.510551,39.652738,3.240709,15.022287,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04-22,54.193806,58.347919,56.883923,14.076679,30.012212,5.887812,14.513246,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-04-29,55.008658,59.394705,58.769437,15.844641,20.426571,8.784216,16.289696,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
plt.figure
fig, ax = plt.subplots(1, 1,figsize=(9,6))
ax_sub = ax.twinx()
l1, = ax.plot(data_all4['粗钢产量同比'], 'pink', label='price')
l2, = ax_sub.plot(data_all4_smoothed1['粗钢产量同比'], 'green', label='total')

ax.set_ylim([-0.2,0.2])
ax_sub.set_ylim([-0.05,0.05])
plt.show()

In [23]:
# 动态因子模型：提取宏观指标共同因子，根据回归系数自动调整因子符号
def Dynamic_Factor_Model(data_dfm, smoothed=True):
    model_dfm = tsa.DynamicFactor(data_dfm, k_factors=1, factor_order=3, error_order=0, error_cov_type="scalar")
    initial_res_dfm = model_dfm.fit(method='powell', disp=False, maxiter=300)
    res_dfm = model_dfm.fit(initial_res_dfm.params, method='lbfgs', disp=False, maxiter=300)
    if smoothed:
        return pd.Series(pd.np.sign(res_dfm.params[:data_dfm.shape[1]].sum())*res_dfm.factors.smoothed[0], index=data_dfm.index)
    else:
        return pd.Series(pd.np.sign(res_dfm.params[:data_dfm.shape[1]].sum()) * res_dfm.factors.filtered[0], index=data_dfm.index)

In [35]:

Y=data_all1['中债国债到期收益率:10年（T+1）']
Y=Y.dropna()
y_small=Y[(Y.index >=pd.to_datetime('20160124')) & (Y.index <= pd.to_datetime(today_time2))]

y_small_diff = y_small.diff(1)
y_small_diff0 = data_all1['中债国债到期收益率：10年'].diff(1)
y_small_diff1 = y_small_diff0.shift(1)
y_small_diff2 = y_small_diff0.shift(2)

alist = ['PMI:主要原材料购进价格','铜现货','螺纹现货','热卷现货','平均批发价:猪肉','PMI','PMI：新订单','PMI：新出口订单','工业增加值：当月同比','出口金额：当月同比','商品房销售面积：累计同比','销量：汽车：当月同比','BDI','CCFI','产量:家用电冰箱:当月同比','工业企业:亏损企业亏损额:累计同比','工业企业:利润总额:累计同比','工业增加值:国有及国有控股企业:当月同比','工业增加值:汽车制造业:当月同比','PMI:从业人员','PMI:生产','PMI:新出口订单','PMI新订单','商品房均价：同比','商品房销售面积：同比','粗钢产量同比','挖掘机产量同比','小松挖掘机使用时长同比','欧美PMI均值','南华工业品指数']
blist = ['社会融资规模存量：同比','社会融资规模存量：人民币贷款：同比','社会融资规模存量：企业债券：同比','M1：同比','M2：同比','社会融资规模:企业债券净融资:当月值:初值','社融-M2','M1同比','地方债净融资额(亿元)','国债净融资额(亿元)']
clist = ['人民币贷款占GDP比','短期人民币贷款占GDP比','中长期人民币贷款占GDP比','居民部门贷款占GDP比','居民部门短期贷款占GDP比','居民部门长期贷款占GDP比','企业贷款占GDP比','企业短期贷款占GDP比','企业中期贷款占GDP比']
dlist = ['存单利率AAA1Y与MLF利率利差','利率互换FR007与国开到期收益率价差1Y','利率互换FR007与国开到期收益率价差5Y','超储','货币当局:政府存款','同业存单利率-政策利率']
elist = ['隐含税率','隔夜成交占比']
flist = ['有效汇率同比','信用利差','沪深300市盈率']
tlist = alist+blist+clist+dlist+elist+flist

def getdata(trainendtime):
    #经济景气度
    data_eco=data_all4_smoothed1[alist]
    data_eco_small=data_eco[(data_eco.index >=pd.to_datetime('20070101')) & (data_eco.index <= pd.to_datetime(trainendtime))]
    data_eco_small_norm=(data_eco_small-data_eco_small.mean())/data_eco_small.std()
    data_eco_small_norm=data_eco_small_norm.fillna(method='ffill').dropna()
    res_dfm_eco= Dynamic_Factor_Model(data_eco_small_norm).rename('经济景气度因子')

    #流动性因子
    data_liq=data_all4_smoothed1[blist]
    data_liq_small=data_liq[(data_liq.index >=pd.to_datetime('20070101')) & (data_liq.index <= pd.to_datetime(trainendtime))]
    data_liq_small_norm=(data_liq_small-data_liq_small.mean())/data_liq_small.std()
    data_liq_small_norm=data_liq_small_norm.fillna(method='ffill').dropna()
    res_dfm_liq= Dynamic_Factor_Model(data_liq_small_norm).rename('供求关系因子')
    
    #信贷规模因子
    data_lev=data_all4_smoothed1[clist]
    data_lev_small=data_lev[(data_lev.index >=pd.to_datetime('20070101')) & (data_lev.index <= pd.to_datetime(trainendtime))]
    data_lev_small_norm=(data_lev_small-data_lev_small.mean())/data_lev_small.std()
    data_lev_small_norm=data_lev_small_norm.fillna(method='ffill').dropna()
    res_dfm_lev= Dynamic_Factor_Model(data_lev_small_norm).rename('供给面因子')
    
    #资金面因子
    data_int=data_all4_smoothed1[dlist]
    data_int_small=data_int[(data_int.index >=pd.to_datetime('20070101')) & (data_int.index <= pd.to_datetime(trainendtime))]
    data_int_small_norm=(data_int_small-data_int_small.mean())/data_int_small.std()
    data_int_small_norm=data_int_small_norm.fillna(method='ffill').dropna()
    res_dfm_int= Dynamic_Factor_Model(data_int_small_norm).rename('资金面因子')
    res_dfm_int = res_dfm_int.dropna()
    
    #情绪因子
    data_mood=data_all4_smoothed1[elist]
    data_mood_small=data_mood[(data_mood.index >=pd.to_datetime('20140720')) & (data_mood.index <= pd.to_datetime(trainendtime))]
    data_mood_small_norm=(data_mood_small-data_mood_small.mean())/data_mood_small.std()
    data_mood_small_norm=data_mood_small_norm.fillna(method='ffill').dropna()
    res_dfm_mood= Dynamic_Factor_Model(data_mood_small_norm).rename('情绪因子') 
    
    #其他价值因子
    data_value=data_all4_smoothed1[flist]
    data_value_small=data_value[(data_value.index >=pd.to_datetime('20140720')) & (data_value.index <= pd.to_datetime(trainendtime))]
    data_value_small_norm=(data_value_small-data_value_small.mean())/data_value_small.std()
    data_value_small_norm=data_value_small_norm.fillna(method='ffill').dropna()
    res_dfm_value= Dynamic_Factor_Model(data_value_small_norm).rename('其他相对价值因子')
    
    X_1 = res_dfm_eco.to_frame()
    #X_2 = res_dfm_inf.to_frame()
    X_3 = res_dfm_liq.to_frame()
    X_4 = res_dfm_lev.to_frame()
    X_5 = res_dfm_int.to_frame()
    X_6 = res_dfm_mood.to_frame()
    X_7 = res_dfm_value.to_frame()
    df_groups=[X_1,X_3,X_4,X_5,X_6,X_7]
    X=reduce(lambda left, right: pd.merge(left, right, how='left', left_index=True, right_index=True),df_groups)

    X=X.dropna()
    #Xdata = sm.add_constant(X)
    X_diff=X.diff(1)
    
    X_diff=pd.merge(X_diff,y_small_diff0,how='left', left_index=True, right_index=True)

    X_data=X_diff[(X_diff.index >= pd.to_datetime('20160124')) & (X_diff.index <= trainendtime)]
    y_small_diff_train = y_small_diff[(y_small_diff.index >= pd.to_datetime('20160124')) & (y_small_diff.index <= pd.to_datetime(trainendtime))]
    return X_data, y_small_diff_train

  

In [36]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.stats.stattools import durbin_watson
%matplotlib inline

In [66]:
trainendtime = pd.to_datetime(today_time2)
X_data, y_small_diff_train = getdata(trainendtime)


In [31]:
X_data.to_excel('X_data.xlsx')

In [67]:
X_data

,经济景气度因子,供求关系因子,供给面因子,资金面因子,情绪因子,其他相对价值因子,中债国债到期收益率：10年
2016-01-24,NaN,NaN,NaN,NaN,NaN,NaN,0.027407
2016-01-31,0.474580,4.548491,1.444092,0.249594,-4.853795,-2.312970,0.075493
2016-02-07,0.206995,4.510566,4.013132,0.274665,-5.885006,-2.405250,-0.003367
2016-02-14,-0.169685,4.467932,6.342571,0.310495,-6.511565,-2.439007,-0.013800
2016-02-21,-0.559095,4.418729,7.194972,0.310834,-6.603410,-2.518123,-0.014267
2016-02-28,-0.807381,4.361981,5.837212,0.311494,-6.150224,-2.526161,0.021100
2016-03-06,-0.729745,4.296596,2.497622,0.280456,-5.265658,-2.569237,0.019600
2016-03-13,-0.161406,4.223119,-1.734388,0.253405,-4.154933,-2.538750,0.011000
2016-03-20,0.974518,4.140976,-5.338100,0.206294,-3.050025,-2.544982,-0.058800
2016-03-27,2.602785,4.051294,-6.954738,0.169700,-2.142760,-2.485384,0.003820


In [68]:
#step1:ADF测试，检验单个变量是否平稳
def adfuller_test(series, signif=0.05, name='', verbose=False):
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")  

In [69]:
df = X_data.copy()
df = df.dropna()

for name, column in df.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')
#结果显示供求关系因子和资金面因子数据非平稳 -- 单个序列非平稳，进行协整检验

    Augmented Dickey-Fuller Test on "经济景气度因子" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -4.0232
 No. Lags Chosen       = 16
 Critical value 1%     = -3.449
 Critical value 5%     = -2.87
 Critical value 10%    = -2.571
 => P-Value = 0.0013. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "供求关系因子" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -2.3781
 No. Lags Chosen       = 17
 Critical value 1%     = -3.449
 Critical value 5%     = -2.87
 Critical value 10%    = -2.571
 => P-Value = 0.148. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "供给面因子" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationar

In [70]:
#step2:协整检验，检验多变量平稳性
#单个序列非平稳，进行协整检验
def cointegration_test(df, alpha=0.05): 
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(df)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
经济景气度因子 ::  363.05    > 111.7797  =>   True
供求关系因子 ::  266.97    > 83.9383   =>   True
供给面因子  ::  175.73    > 60.0627   =>   True
资金面因子  ::  103.4     > 40.1749   =>   True
情绪因子   ::  60.75     > 24.2761   =>   True
其他相对价值因子 ::  30.26     > 12.3212   =>   True
中债国债到期收益率：10年 ::  8.4       > 4.1296    =>   True


In [71]:
#step3：进行Granger’s Causality Test ， 检验不同序列之间存在互相影响
df = X_data.copy()
df = df.dropna()
maxlag=12
test='ssr_chi2test'
variables=df.columns
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(df, variables = df.columns)

,经济景气度因子_x,供求关系因子_x,供给面因子_x,资金面因子_x,情绪因子_x,其他相对价值因子_x,中债国债到期收益率：10年_x
经济景气度因子_y,1.0000,0.0000,0.0000,0.0005,0.0048,0.0406,0.0075
供求关系因子_y,0.0000,1.0000,0.0012,0.0001,0.0702,0.0000,0.0609
供给面因子_y,0.0001,0.0015,1.0000,0.3050,0.0002,0.0006,0.1714
资金面因子_y,0.1951,0.0000,0.3005,1.0000,0.0000,0.0000,0.0094
情绪因子_y,0.0070,0.0320,0.0001,0.1214,1.0000,0.4790,0.0073
其他相对价值因子_y,0.0043,0.0000,0.0000,0.0000,0.0439,1.0000,0.1157
中债国债到期收益率：10年_y,0.0069,0.2081,0.2098,0.0000,0.0018,0.1598,1.0000


In [80]:
# step4：划分训练集和测试集
# 最后10个时间点作为测试集
nobs = 10
df_train, df_test = df[0:-nobs], df[-nobs:]

In [81]:
# step5：使用VAR之间，先差分处理使单个变量变得平稳

df_differenced = df_train.diff().dropna()
for name, column in df_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "经济景气度因子" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -5.8517
 No. Lags Chosen       = 15
 Critical value 1%     = -3.449
 Critical value 5%     = -2.87
 Critical value 10%    = -2.571
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "供求关系因子" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -3.8816
 No. Lags Chosen       = 17
 Critical value 1%     = -3.449
 Critical value 5%     = -2.87
 Critical value 10%    = -2.571
 => P-Value = 0.0022. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "供给面因子" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level  

In [82]:
# step6：选择模型阶数并训练，根据BIC值，lag=6时达到局部最优
model = VAR(df_differenced)
for i in [1,2,3,4,5,6,7,8,9]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('BIC : ', result.bic, '\n')

Lag Order = 1
BIC :  -32.544982260710185 

Lag Order = 2
BIC :  -45.48520801482779 

Lag Order = 3
BIC :  -54.67272928038965 

Lag Order = 4
BIC :  -62.42272155511158 

Lag Order = 5
BIC :  -64.23222838019586 

Lag Order = 6
BIC :  -64.89514747825812 

Lag Order = 7
BIC :  -64.36838373983242 

Lag Order = 8
BIC :  -64.17172915592747 

Lag Order = 9
BIC :  -63.57275000519849 



In [83]:
#如上所示选择lag=6拟合模型
model_fitted = model.fit(6)
#model.fitted.summary()

In [84]:
# step8：durbin watson test，检验残差项中是否还存在相关性，这一步的目的是确保模型已经解释了数据中所有的方差和模式
out = durbin_watson(model_fitted.resid)
for col, val in zip(df.columns, out):
    print(col,':',round(val, 2)) 
# 检验值越接近2，说明模型越好；

经济景气度因子 : 1.95
供求关系因子 : 1.91
供给面因子 : 1.95
资金面因子 : 1.88
情绪因子 : 1.98
其他相对价值因子 : 1.95
中债国债到期收益率：10年 : 2.04


In [85]:
# step9：模型预测
lag_order = model_fitted.k_ar
forecast_input = df_differenced.values[-lag_order:]
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df.index[-nobs:], columns=df.columns + '_1d')
df_forecast

,经济景气度因子_1d,供求关系因子_1d,供给面因子_1d,资金面因子_1d,情绪因子_1d,其他相对价值因子_1d,中债国债到期收益率：10年_1d
2023-02-19,1.185638,0.074823,1.695958,-0.047694,0.450511,0.159340,0.051801
2023-02-26,1.345064,0.074928,-0.784868,-0.038586,0.413332,0.102378,-0.009850
2023-03-05,1.395170,0.073345,-3.096408,-0.018864,0.256089,0.131083,-0.013574
2023-03-12,1.287293,0.072171,-4.098098,-0.018975,-0.007840,0.076286,-0.019774
2023-03-19,0.999639,0.070043,-3.270917,-0.006048,-0.335622,0.102415,-0.011975
2023-03-26,0.547373,0.068297,-0.970039,-0.014592,-0.656323,0.048934,0.000559
2023-04-02,-0.015096,0.065686,1.736340,-0.004625,-0.886532,0.071767,0.009229
2023-04-09,-0.599652,0.063035,3.602460,-0.014044,-0.957487,0.020099,0.019921
2023-04-16,-1.099625,0.059403,3.800565,0.001334,-0.839579,0.040614,0.006821
2023-04-23,-1.413147,0.055548,2.315084,-0.001395,-0.556887,-0.008092,0.006504


In [94]:
# step10：将差分后的值还原为原数据
def invert_transformation(df_train, df_forecast):
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

In [95]:

df_results = invert_transformation(df_train, df_forecast) 
df_results.loc[:, ['经济景气度因子_forecast', '供求关系因子_forecast', '供给面因子_forecast', '资金面因子_forecast', '情绪因子_forecast', '其他相对价值因子_forecast','中债国债到期收益率：10年_forecast']]

,经济景气度因子_forecast,供求关系因子_forecast,供给面因子_forecast,资金面因子_forecast,情绪因子_forecast,其他相对价值因子_forecast,中债国债到期收益率：10年_forecast
2023-02-19,-0.001751,-0.555242,9.618805,0.293455,-0.616935,0.261352,0.046701
2023-02-26,1.343313,-0.480313,8.833937,0.254869,-0.203603,0.363730,0.036851
2023-03-05,2.738483,-0.406969,5.737528,0.236005,0.052486,0.494813,0.023277
2023-03-12,4.025776,-0.334798,1.639430,0.217030,0.044646,0.571098,0.003503
2023-03-19,5.025414,-0.264755,-1.631487,0.210982,-0.290976,0.673513,-0.008472
2023-03-26,5.572787,-0.196458,-2.601526,0.196389,-0.947298,0.722447,-0.007913
2023-04-02,5.557691,-0.130772,-0.865186,0.191765,-1.833830,0.794214,0.001317
2023-04-09,4.958039,-0.067737,2.737275,0.177721,-2.791317,0.814313,0.021237
2023-04-16,3.858413,-0.008334,6.537839,0.179055,-3.630895,0.854927,0.028058
2023-04-23,2.445266,0.047214,8.852924,0.177660,-4.187782,0.846835,0.034562


In [ ]:
#模型评估
def reg(X_data, y_small_diff_train):
    
    
    X2 = X_data.iloc[:-1,:]  # 训练集X
    X3 = X_data.iloc[-1:,:]  # 测试集X，1维
  
    Modeldata = pd.merge(X2,y_small_diff_train,how='left', left_index=True, right_index=True).dropna()
    
    X2 = Modeldata.iloc[:,:-1]
    y_small_diff_train = Modeldata.iloc[:,-1]

    est = sm.OLS(y_small_diff_train, X2).fit()
    y1 = est.predict(X3)
    return est, y1

trainendtime = pd.to_datetime('20191130')
endtime = pd.to_datetime(today_time2)
Y1 = pd.DataFrame(columns = ["Y1"],dtype=object)
while trainendtime <= endtime + pd.DateOffset(weeks=1):
    X_data, y_small_diff_train = getdata(trainendtime)
    est, y1 = reg(X_data, y_small_diff_train)
    Y1 = Y1.append(pd.DataFrame(y1.values,index=y1.index,columns=['Y1']))
    trainendtime += pd.DateOffset(weeks=1)

print(Y1)
est.summary()